In [1]:
! pip3 install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/74/07/edce54779f5c3fe8ab8390eafad3d7c8190fce68f922a254ea77f4a94a99/torch-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/20/ac/ab6f42af83349e679b03c9bb18354740c6b58b17dba329fb408730230584/torchvision-0.16.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/11/30/715101782513f94c834ebe3afb9a29b0fae1121f64963db9d39fb80da53e/torchaudio-2.1.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.2/192.3 MB 6.3 MB/s eta 0:00:31
   ---------------------------------------- 0.7/192.3 MB 7.4 MB/s eta 0:00:26
   ---------------------------------------- 1.5/192.3 MB 11.6 MB/s eta 0:00:17
    --------------------------------------- 2.5/192.3 MB 13

In [1]:
# Install the Hugging Face Transformers library and its dependencies
! pip install -U transformers
! pip install -U datasets
! pip install -U sentencepiece

# Import the necessary modules
import os
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, QuestionAnsweringPipeline

# Load the transcripts from the csv file named "cleaned_output_file"
transcripts = pd.read_csv('cleaned_output_file.csv')

# Convert the transcripts dataframe to a dictionary with id as the key and text as the value
transcripts = transcripts.set_index('id')['text'].to_dict()

# Load the test data from the csv file named "test.csv"
test = pd.read_csv('test.csv')

# Load the pre-trained tokenizer and model for question answering
# You can choose any model from https://huggingface.co/models?filter=question-answering
# For example, you can use dmis-lab/biobert-v1.1
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-v1.1")

# Create a question answering pipeline using the tokenizer and model
qa_pipeline = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

# Initialize an empty list to store the answers
answers = []

# Loop over all the rows in the test data
for _, row in test.iterrows():
    # Get the question and the transcript id from the row
    question = row['Question']
    transcript_id = row['Transcript']

    # Get the transcript text from the transcripts dictionary
    transcript = transcripts[transcript_id]

    # Use the question answering pipeline to generate an answer based on the question and the transcript
    # Add a try-except block to catch and handle any errors or exceptions
    try:
        answer = qa_pipeline(question=question, context=transcript)
    except Exception as e:
        # Print the error message and the question id to the console
        print(f"Error: {e} for question id {row['Id']}")
        # Set the answer to None
        answer = None

    # Append the answer and the question id to the answers list
    if answer is not None:
        answers.append([row['Id'], answer['answer']])
    else:
        answers.append([row['Id'], ""])

# Convert the answers list to a pandas dataframe
answers = pd.DataFrame(answers, columns=["Id", "Text"])
answers.to_csv("sample_submission_10nov_up8.csv", index=False)  # Removed index and index_label


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [2]:
  # Append the answer and the question id to the answers list
if answer is not None:
    answers.append([row['Id'], answer['answer']])
else:
    answers.append([row['Id'], ""])

# Convert the answers list to a pandas dataframe
answers = pd.DataFrame(answers, columns=["Id", "Text"])
answers.to_csv("sample_submission_10nov_up2.csv", index=False)  # Removed index and index_label

In [6]:
# Import pandas library
import pandas as pd

# Read the csv files as dataframes
df1 = pd.read_csv("llama_submission_02.csv")
df2 = pd.read_csv("Submission_up7_test.csv")

# Merge the dataframes on id column
df = pd.merge(df1, df2, on="Id")

# Define a function to replace "none" with the corresponding text from df2
def replace_none(row):
    if row["text_x"] == "None":
        return row["text_y"]
    else:
        return row["text_x"]

# Apply the function to the merged dataframe
df["Text"] = df.apply(replace_none, axis=1)

# Drop the unnecessary columns
df = df.drop(["text_x", "text_y"], axis=1)

# Write the modified dataframe to a new csv file
df.to_csv("Final_Submission_D4G.csv", index=False)
